# Title
[]()

In [2]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [3]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
# from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict, create_text_dict_from_folder

In [116]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [5]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

# Create text_dict

In [10]:
folder_path = '../text/2023-04-14'
encoding='ISO-8859-1'
subset=None
# subset=300

text_dict = create_text_dict_from_folder(folder_path, encoding='ISO-8859-1', subset=subset)
text_dict

{1: 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during middl

# Create prompts

In [11]:
experiment_num = 1

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

summarize_task = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]

simple_simplify_task = [
    # "Use terms a 12-year-old can understand.",
    "Assume your audience has no science background.",
    "Use a fun tone."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

no_audience = ['']

user_simplify_task = [
    "Use language appropriate for",
    # "Use terms a 12-year-old can understand.",
]

simplify_audience = [
    # "",
    "a lay audience",
    "people without a science background",
]

user_relevance_task = [
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
]

relevance_audience = [
    # "lay audience",
    # "",
    "seniors",
    "people who enjoy sports"
]



# working scripts from last notebook

In [16]:
import time
from itertools import product
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict




## Initialize parameters and variables

In [13]:
iteration_id = 1
n_choices = 5
qna_dict = dict()
chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()


In [18]:
summary_iteration_id = iteration_id
pause_per_request=0

# Run it: Iteration 1

## Create Initial Summary

In [17]:
qna_dict, chaining_dict = batch_summarize_chain(
    text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id
    )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (20, 9)


In [19]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younger individua..."
1,2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A recent study found no differences in recovery response between young and middle-aged men following resistance exercise. Additionally, there were no differences in inflammatory or muscle damage response between the two groups. The study suggests that changes in muscle performance may begin duri..."
2,3,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A study comparing young and middle-aged men in the recovery response from resistance exercise found no significant differences in performance measures, muscle pain or soreness, inflammatory or muscle damage response. Differences in strength levels were observed between age groups, but recovery f..."
3,4,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"""New study finds no difference in recovery response from resistance exercise between young and middle-aged men! Results showed similar changes in performance and markers of muscle damage and inflammation, suggesting that middle-aged adults who engage in resistance training are not at increased r..."
4,5,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance E

In [20]:
simplify_prompts = user_simplify_task
audience = simplify_audience
simple_summaries = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 1
n_choices: 1
**text1_prompt00
simplify_iteration:  1
Task: Use language appropriate for a lay audience
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04', 'response_05']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_04
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_05
	Done creating pro

## simple summary dataframe

In [21]:
chatbot_id = iteration_id

simple_summaries_dict = process_chaining_results(
    simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
    results_type='simple', chatbot_id=chatbot_id, 
    save_df=True, save_chatbot=False
    )
simple_summaries_dict[chatbot_id]

New results shape: (40, 6)
simple summaries dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/prompt_chain_simple_summaries_2023-04-19_1137.sav
Time completed: 2023-04-19 11:37:22.991864
	Object saved as pickle
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/prompt_chain_simple_summaries_2023-04-19_1137.csv
Time completed: 2023-04-19 11:37:23.132923
	Dictionary converted to CSV


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,simple summary choice,audience,simplify task,full simplify task,simple summary
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younger individua...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A recent study found that middle-aged men who regularly do resistance training don't have a higher risk of muscle soreness or damage compared to younger men. This means that if you're middle-aged and you work out regularly, you don't have to worry about hurting your muscles more than younger peo..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A recent study found no differences in recovery response between young and middle-aged men following resistance exercise. Additionally, there were no differences in inflammatory or muscle damage response between the two groups. The study suggests that changes in muscle performance may begin duri...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A new study found that young and middle-aged men have similar recovery responses after doing weightlifting exercises. The study also found that there were no differences in how their muscles responded to the exercise. This means that starting in middle age, our muscles may start to change, but d..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,3,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A study comparing young and middle-aged men in the recovery response from resistance exercise found no significant differences in performance measures, muscle pain or soreness, inflammatory or muscle damage response. Differences in strength levels were observed between age groups, but recovery f...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A recent study looked at how young and middle-aged men recover from exercise. The researchers found that there were no big differences in how the two groups felt after 

In [22]:
relevance_prompts = user_relevance_task
relevance = prompt_chaining_dict(relevance_prompts, relevance_audience, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id, prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )


summary_iteration_id: 1
n_choices: 1
**text1_prompt00
relevance_iteration:  1
Task: Add 1-2 sentences to make this relevant for seniors
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04', 'response_05']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_04
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	..

In [23]:
chatbot_id = iteration_id
relevance_dict = process_chaining_results(
    relevance_dict, qna_dict, chatbot_dict, iteration_id, 
    results_type='relevance', chatbot_id=chatbot_id, 
    save_df=True, save_chatbot=True
    )

New results shape: (40, 6)
added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/prompt_chain_simple_summaries_2023-04-19_1141.sav
Time completed: 2023-04-19 11:41:26.788252
	Object saved as pickle
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/prompt_chain_simple_summaries_2023-04-19_1141.csv
Time completed: 2023-04-19 11:41:26.805281
	Dictionary converted to CSV
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chaining_bot_2023-04-19_1141.sav
Time completed: 2023-04-19 11:41:26.834831
	Object saved as pickle


## Relevance dataframe

In [158]:

relevance_dict[chatbot_id]

,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,relevance choice,audience,relevance task,full relevance task,relevance statement
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors who regularly engage in resistance training, this study suggests that they may not e..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors, this study highlights the importance of engaging in resistance training to maintain..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study comparing young and middle-aged men in the recovery response from resistance exercise fo...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,This study is particularly relevant for seniors as it suggests that recreational resistance trai...
3,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"""New study finds no difference in recovery response from resistance exercise between young and m...",1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors, resistance training can be an effective way to maintain muscle mass and strength, w..."
4,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,5,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in the recovery response between young and middle-aged men a...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors, it is important to note that participating in recreational resistance training can ..."
5,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Respon

In [176]:
print(relevance_dict[chatbot_id].shape)
[column for column in relevance_dict[chatbot_id].columns]

(40, 14)


['article_title',
 'choice',
 'system_role',
 'model',
 'text',
 'prep step',
 'summarization task',
 'full summarization task',
 'summary',
 'relevance choice',
 'audience',
 'relevance task',
 'full relevance task',
 'relevance statement']

In [287]:
test_df = relevance_dict[chatbot_id].copy().head()#
test_df.insert(loc=-1, column='la', value=9)
test_df

ValueError: unbounded slice

In [232]:
pivot_df = relevance_dict[chatbot_id].pivot(
    columns=[
        'audience', 
        # 'relevance task'
        ],
    values='relevance statement',
    index=['summary', 'relevance task']
    ).reset_index()
print(pivot_df.shape, pivot_df.columns)
pivot_df

(20, 4) Index(['summary', 'relevance task', 'people who enjoy sports', 'seniors'], dtype='object', name='audience')


audience,summary,relevance task,people who enjoy sports,seniors
0,"""New study finds no difference in recovery response from resistance exercise between young and m...",Add 1-2 sentences to make this relevant for,"This study is particularly relevant for athletes who engage in resistance training, as it sugges...","For seniors, resistance training can be an effective way to maintain muscle mass and strength, w..."
1,A recent study found no differences in recovery response between young and middle-aged men follo...,Add 1-2 sentences to make this relevant for,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...","For seniors, this study highlights the importance of engaging in resistance training to maintain..."
2,A recent study found no differences in the recovery response between young and middle-aged men a...,Add 1-2 sentences to make this relevant for,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...","For seniors, it is important to note that participating in recreational resistance training can ..."
3,A recent study investigated the effects of aging on muscle soreness and damage after an intensiv...,Add 1-2 sentences to make this relevant for,This study is great news for master athletes who may have been concerned about their ability to ...,"This study is particularly relevant for seniors who engage in intensive exercise protocols, as i..."
4,A recent study investigated the effects of aging on muscle soreness and damage following an inte...,Add 1-2 sentences to make this relevant for,This study provides valuable insights for athletes of all ages who engage in intensive squat exe...,This study is particularly relevant for seniors who are interested in maintaining their physical...
5,A recent study investigated the effects of aging on muscle soreness and damage following an inte...,Add 1-2 sentences to make this relevant for,This study provides valuable insights for athletes of all ages who engage in intensive squat exe...,For seniors who are concerned about muscle soreness and damage following an intensive exercise r...
6,A recent study investigated the effects of aging on muscle soreness and muscle damage following ...,Add 1-2 sentences to make this relevant for,"For sports enthusiasts, this study provides valuable insights into the effects of aging on muscl...",This study is particularly relevant for seniors who engage in resistance exercise as it suggests...
7,A study comparing young and middle-aged men in the recovery response from resistance exercise fo...,Add 1-2 sentences to make this relevant for,"For sports enthusiasts, this study suggests that age should not be a limiting factor in recovery...",This study is particularly relevant for seniors as it suggests that recreational resistance trai...
8,A study found no differences in recovery response between young and middle-aged men after high-v...,Add 1-2 sentences to make this relevant for,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...","For seniors who regularly engage in resistance training, this study suggests that they may not e..."
9,Hey! I just read a study on the recovery response from resistance exercise between young and mid...,Add 1-2 sentences to make this relevant for,"For athletes and fitness enthusiasts, this study highlights the importance of resistance trainin...",This study highlights the importance of resistance training for seniors as it may help mitigate ...


In [26]:
relevance_dict[chatbot_id]['full relevance task'].unique()

array(['Add 1-2 sentences to make this relevant for seniors',
       'Add 1-2 sentences to make this relevant for people who enjoy sports'],
      dtype=object)

# Fix saving error
Previous version saved wrong objects to file

In [49]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description=f'chaining_bot_{results_type}',
                csv_path=None, pickle_path=pickle_path)
            print('Chaining object (chatbot) pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

In [50]:
chatbot_id = iteration_id
relevance_dict = process_chaining_results(
    relevance_dict, qna_dict, chatbot_dict, iteration_id, 
    results_type='relevance', chatbot_id=chatbot_id, 
    save_df=True, save_chatbot=True
    )

New results shape: (40, 6)
added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/prompt_chain_simple_summaries_relevance_2023-04-19_1159.sav
Time completed: 2023-04-19 11:59:01.193342
	Object saved as pickle
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/prompt_chain_simple_summaries_relevance_2023-04-19_1159.csv
Time completed: 2023-04-19 11:59:01.221380
	DataFrame saved as CSV
DataFrame pickled.
Unable to save pickle
Chaining object (chatbot) pickled.


### save error Iteration 2

In [ ]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_output(
                chatbot_dict[iteration_id], description=f'chaining_bot_{results_type}',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

In [51]:
results_type='relevance'
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'

save_output(
    chatbot_dict[iteration_id], description=f'chaining_bot_{results_type}',
    csv_path=None, pickle_path=pickle_path)

Unable to save pickle


In [53]:
type(chatbot_dict[iteration_id])

dict

In [54]:
chatbot_dict

{1: {'text1_prompt00': <__main__.Chaining at 0x161553ccd90>,
  'text1_prompt01': <__main__.Chaining at 0x161777f5280>,
  'text2_prompt00': <__main__.Chaining at 0x161552bc880>,
  'text2_prompt01': <__main__.Chaining at 0x16176ab2e80>}}

### Error message when trying to pickle the class instance

In [57]:
def save_chatbot(model, filename, ext='sav', path=None,append_version=False):
    """
    Export object as a pickle.
    Parameters:
    - model: Model variable name.
    - filename: Root of the filename.
    - extension: Extension to append (do not include dot as it will be added)
    - filepath (raw string): Use the format r'<path>'. If None, file is saved in same director.
    - append_version (bool): If true, append date and time to end of filename.
    """
    if path:
        path = f'{path}/'.replace('\\','/')
    if append_version == True:
        filename+=datetime.now().strftime('%Y-%m-%d_%H%M')
    with open (path+filename+'.'+ext, 'wb') as fh:
        pickle.dump(model, fh)
    print('File saved: ',path+filename+'.'+ext)
    print('Time completed:', datetime.now())


filename = 'chatbot_2023-04-19_1159'
save_chatbot(chatbot_dict[iteration_id], filename, path=pickle_path)

PicklingError: Can't pickle <class '__main__.Chaining'>: it's not the same object as __main__.Chaining

In [60]:
filename_root = 'chatbot_2023-04-19_1159'
for key, item in chatbot_dict[iteration_id].items():
    filename = f'{filename_root}_{key}'
    save_chatbot(item, filename, path=pickle_path)

PicklingError: Can't pickle <class '__main__.Chaining'>: it's not the same object as __main__.Chaining

In [64]:
chatbot_dict[iteration_id]['text1_prompt00'].qna

{'article_title': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men',
 'system_role': 'You are an expert at science communication.',
 'model': 'gpt-3.5-turbo',
 'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young 

In [65]:
chatbot_dict[iteration_id]['text1_prompt00'].relevance_dict

{1: {'response_01': {0: {'relevance choice': 1,
    'relevance task': 'Add 1-2 sentences to make this relevant for',
    'audience': 'seniors',
    'full relevance task': 'Add 1-2 sentences to make this relevant for seniors',
    'relevance statement': 'For seniors who regularly engage in resistance training, this study suggests that they may not experience increased muscle soreness or damage compared to younger individuals. However, it is important to note that changes in muscle performance may begin during middle age, so it is recommended that seniors continue to engage in multijoint, dynamic constant resistance exercises to maintain their muscle strength and overall fitness.',
    'preceding summary': 'A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younge

In [77]:
chatbot_dict[iteration_id]['text1_prompt00']

In [66]:
filename_root = 'chatbot_2023-04-19_1159_relevance'
for key, item in chatbot_dict[iteration_id].items():
    filename = f'{filename_root}_{key}'
    save_chatbot(item.relevance_dict, filename, path=pickle_path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatbot_2023-04-19_1159_relevance_text1_prompt00.sav
Time completed: 2023-04-19 12:17:45.695116
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatbot_2023-04-19_1159_relevance_text1_prompt01.sav
Time completed: 2023-04-19 12:17:45.698115
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatbot_2023-04-19_1159_relevance_text2_prompt00.sav
Time completed: 2023-04-19 12:17:45.702117
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatbot_2023-04-19_1159_relevance_text2_prompt01.sav
Time completed: 2023-04-19 12:17:45.705119


In [67]:
loadpickle('chatbot_2023-04-19_1159_relevance_text1_prompt00.sav', pickle_path)

Time completed: 2023-04-19 12:42:19.867578


{1: {'response_01': {0: {'relevance choice': 1,
    'relevance task': 'Add 1-2 sentences to make this relevant for',
    'audience': 'seniors',
    'full relevance task': 'Add 1-2 sentences to make this relevant for seniors',
    'relevance statement': 'For seniors who regularly engage in resistance training, this study suggests that they may not experience increased muscle soreness or damage compared to younger individuals. However, it is important to note that changes in muscle performance may begin during middle age, so it is recommended that seniors continue to engage in multijoint, dynamic constant resistance exercises to maintain their muscle strength and overall fitness.',
    'preceding summary': 'A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younge

In [84]:
filename_root = 'chatbot_2023-04-19_1159_relevance'
for key, item in chatbot_dict[iteration_id].items():
    print(key)
    for attr, value in vars(item).items():
        print(f'\t{attr}')

text1_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text1_prompt01
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text2_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text2_prompt01
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict


In [85]:
type(chatbot_dict[iteration_id])

dict

### Iteration 3: Able to save instance as dictionary as picle

In [92]:
def save_instance_to_dict(chatbot_dict_iteration, filename=None, description='batch_Chaining_attributes', 
        ext='sav', append_version=True, 
        path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Export object as a pickle.
    Parameters:
    - model: Model variable name.
    - filename: Root of the filename.
    - extension: Extension to append (do not include dot as it will be added)
    - filepath (raw string): Use the format r'<path>'. If None, file is saved in same director.
    - append_version (bool): If true, append date and time to end of filename.
    """
    chatbot_dictionary = {}


    for key, item in chatbot_dict_iteration.items():
        chatbot_dictionary[key] = dict()
        print(key)
        for attr, value in vars(item).items():
            print(f'\t{attr}')
            chatbot_dictionary[key][attr] = value
    try:
        save_output(
            chatbot_dictionary, description=description,
            append_version=append_version, csv_path=None, pickle_path=path
        )
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        print(f'Unable to save chatbot dictionary')
    print('Dictionary keys:', chatbot_dictionary.keys())
    return chatbot_dictionary

filename = 'chatbot_2023-04-19_1159'

# chatbot_dictionary = save_instance_to_dict(chatbot_dict[iteration_id], filename)
save_instance_to_dict(chatbot_dict[iteration_id], filename)

text1_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text1_prompt01
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text2_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text2_prompt01
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/batch_Chaining_attributes_2023-04-19_1336.sav
Time completed: 2023-04-19 13:36:49.203268
	Object saved as pickle
Dictionary keys: dict_keys(['text1_prompt00', 'text1_prompt01', 'text2_prompt00', 'text2_prompt01'])


{'text1_prompt00': {'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength 

#### Confirm dictionary can be loaded from pickle

In [93]:
loadpickle('batch_Chaining_attributes_2023-04-19_1336.sav', pickle_path)

Time completed: 2023-04-19 13:37:39.768755


{'text1_prompt00': {'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength 

## Iteration 4: Also save as a json file

In [99]:
import json

def save_to_json(obj, filename=None, description='output_dictionary', append_version=False,
    path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Save Python object as a JSON file.
    Parameters:
    - obj: Python object to be saved.
    - filename: Root of the filename.
    - path (raw string): Use the format r'<path>'. If None, file is saved in same directory as script.
    - append_version (bool): If true, append date and time to end of filename.
    """
    if description:
        filename = f'{description}_{datetime.now().strftime("%Y-%m-%d_%H%M")}'
        append_version = False
    elif filename == None:
        filename = f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_outputs'
        append_version = False
    if path:
        path = f'{path}/'.replace('\\','/')
    if append_version:
        filename += f'_{datetime.now().strftime("%Y-%m-%d_%H%M%S")}'
    filename += '.json'
    with open(path+filename, 'w') as f:
        json.dump(obj, f)
    print(f'Object saved as JSON: {filename}')

def save_instance_to_dict(chatbot_dict_iteration, filename=None, description='batch_Chaining_attributes', 
        ext='sav', save_json=True, append_version=True, 
        path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Export object as a pickle.
    Parameters:
    - model: Model variable name.
    - filename: Root of the filename.
    - extension: Extension to append (do not include dot as it will be added)
    - filepath (raw string): Use the format r'<path>'. If None, file is saved in same director.
    - append_version (bool): If true, append date and time to end of filename.
    """
    chatbot_dictionary = {}


    for key, item in chatbot_dict_iteration.items():
        chatbot_dictionary[key] = dict()
        print(key)
        for attr, value in vars(item).items():
            print(f'\t{attr}')
            chatbot_dictionary[key][attr] = value
    if ext:
        try:
            save_output(
                chatbot_dictionary, filename=filename, description=description,
                append_version=append_version, csv_path=None, pickle_path=path
            )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to pickle chatbot dictionary')
        print('Dictionary keys:', chatbot_dictionary.keys())
    if save_json==True:
        try:
            save_to_json(
                chatbot_dictionary, 
                filename=filename, description=description,
                append_version=append_version, path=path
            )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save chatbot dictionary to JSON')
    return chatbot_dictionary

filename = 'chatbot_2023-04-19_1159'

chatbot_dictionary = save_instance_to_dict(chatbot_dict[iteration_id])
# save_instance_to_dict(chatbot_dict[iteration_id], filename)

text1_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text1_prompt01
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text2_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text2_prompt01
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/batch_Chaining_attributes_2023-04-19_1401.sav
Time completed: 2023-04-19 14:01:23.219975
	Object saved as pickle
Dictionary keys: dict_keys(['text1_prompt00', 'text1_prompt01', 'text2_prompt00', 'text2_prompt01'])
Object saved as JSON: batch_Chaining_attributes_2023-04-19_1401.jso

## Iteration 5

In [103]:
import json

def save_to_json(obj, filename=None, description='output_dictionary', append_version=False,
    path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Save Python object as a JSON file.
    Parameters:
    - obj: Python object to be saved.
    - filename: Root of the filename.
    - path (raw string): Use the format r'<path>'. If None, file is saved in same directory as script.
    - append_version (bool): If true, append date and time to end of filename.
    """
    if description:
        filename = f'{description}_{datetime.now().strftime("%Y-%m-%d_%H%M")}'
        append_version = False
    elif filename == None:
        filename = f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_outputs'
        append_version = False
    if path:
        path = f'{path}/'.replace('\\','/')
    if append_version:
        filename += f'_{datetime.now().strftime("%Y-%m-%d_%H%M%S")}'
    filename += '.json'
    with open(path+filename, 'w') as f:
        json.dump(obj, f)
    print(f'Object saved as JSON: {filename}')

def save_instance_to_dict(chatbot_dict_iteration, filename=None, description='batch_Chaining_attributes', 
        append_version=True, 
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles', 
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Export object as a pickle.
    Parameters:
    - model: Model variable name.
    - filename: Root of the filename.
    - extension: Extension to append (do not include dot as it will be added)
    - filepath (raw string): Use the format r'<path>'. If None, file is saved in same director.
    - append_version (bool): If true, append date and time to end of filename.
    """
    chatbot_dictionary = {}


    for key, item in chatbot_dict_iteration.items():
        chatbot_dictionary[key] = dict()
        print(key)
        print('\tAttributes: ', end=', ')
        for attr, value in vars(item).items():
            print(f'{attr}', end=', ')
            chatbot_dictionary[key][attr] = value
        print()
    if pickle_path:
        try:
            save_output(
                chatbot_dictionary, filename=filename, description=description,
                append_version=append_version, csv_path=None, pickle_path=pickle_path
            )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to pickle chatbot dictionary')
        print('Dictionary keys:', chatbot_dictionary.keys())
    if json_path:
        try:
            save_to_json(
                chatbot_dictionary, 
                filename=filename, description=description,
                append_version=append_version, path=json_path
            )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save chatbot dictionary to JSON')
    return chatbot_dictionary

filename = 'batch_Chaining_attributes_2023-04-19_1159'

chatbot_dictionary = save_instance_to_dict(chatbot_dict[iteration_id], append_version=False)
# save_instance_to_dict(chatbot_dict[iteration_id], filename)

text1_prompt00
	Attributes: , text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text1_prompt01
	Attributes: , text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text2_prompt00
	Attributes: , text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text2_prompt01
	Attributes: , text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/batch_Chaining_attributes_2023-04-19_1411.sav
Time completed: 2023-04-19 14:11:17.065364
	Object saved as pickle
Dictionary keys: dict_keys(['text1_prompt00', 'text1_prompt01', 'text2_prompt00', 'text2_prompt01'])
Obj

## Iteration 6: Final version of save functions

In [106]:
import json

def save_to_json(obj, filename=None, description='output_dictionary', append_version=False,
    path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Save Python object as a JSON file.
    Parameters:
    - obj: Python object to be saved.
    - filename: Root of the filename.
    - path (raw string): Use the format r'<path>'. If None, file is saved in same directory as script.
    - append_version (bool): If true, append date and time to end of filename.
    """
    if description:
        filename = f'{description}_{datetime.now().strftime("%Y-%m-%d_%H%M")}'
        append_version = False
    elif filename == None:
        filename = f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_outputs'
        append_version = False
    if path:
        path = f'{path}/'.replace('\\','/')
    if append_version:
        filename += f'_{datetime.now().strftime("%Y-%m-%d_%H%M%S")}'
    filename += '.json'
    with open(path+filename, 'w') as f:
        json.dump(obj, f)
    print(f'Object saved as JSON: {path+filename}')
    print('\tTime completed:', datetime.now())

def save_instance_to_dict(chatbot_dict_iteration, filename=None, description='batch_Chaining_attributes', 
        append_version=True, 
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles', 
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Export object as a pickle.
    Parameters:
    - model: Model variable name.
    - filename: Root of the filename.
    - extension: Extension to append (do not include dot as it will be added)
    - filepath (raw string): Use the format r'<path>'. If None, file is saved in same director.
    - append_version (bool): If true, append date and time to end of filename.
    """
    chatbot_dictionary = {}


    for key, item in chatbot_dict_iteration.items():
        chatbot_dictionary[key] = dict()
        print(key)
        print('\tAttributes: ', end='')
        for attr, value in vars(item).items():
            print(f'{attr}', end=', ')
            chatbot_dictionary[key][attr] = value
        print()
    print('Dictionary keys:', chatbot_dictionary.keys())
    if pickle_path:
        try:
            save_output(
                chatbot_dictionary, filename=filename, description=description,
                append_version=append_version, csv_path=None, pickle_path=pickle_path
            )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to pickle chatbot dictionary')
    if json_path:
        try:
            save_to_json(
                chatbot_dictionary, 
                filename=filename, description=description,
                append_version=append_version, path=json_path
            )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save chatbot dictionary to JSON')
    return chatbot_dictionary

filename = 'batch_Chaining_attributes_2023-04-19_1159'

chatbot_dictionary = save_instance_to_dict(chatbot_dict[iteration_id], append_version=False)
# save_instance_to_dict(chatbot_dict[iteration_id], filename)

text1_prompt00
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text1_prompt01
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text2_prompt00
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text2_prompt01
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
Dictionary keys: dict_keys(['text1_prompt00', 'text1_prompt01', 'text2_prompt00', 'text2_prompt01'])
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/batch_Chaining_attributes_2023-04-19_1414.sav
Time completed: 2023-04-19 14:14:34.874468
	Object saved as pickle
Object save

# Run Iteration 1.1: Save entire dictionary of chatbot objects

In [107]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

In [108]:
chatbot_id = iteration_id
relevance_dict = process_chaining_results(
    relevance_dict, qna_dict, chatbot_dict, iteration_id, 
    results_type='relevance', chatbot_id=chatbot_id, 
    save_df=True, save_chatbot=True
    )

New results shape: (40, 6)
added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/prompt_chain_simple_summaries_relevance_2023-04-19_1437.sav
Time completed: 2023-04-19 14:37:31.346772
	Object saved as pickle
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/prompt_chain_simple_summaries_relevance_2023-04-19_1437.csv
Time completed: 2023-04-19 14:37:31.395560
	DataFrame saved as CSV
DataFrame pickled.
Saving Chaining object (chatbot)...
text1_prompt00
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text1_prompt01
	Attributes: text, 

In [109]:
chatbot_id = iteration_id

simple_summaries_dict = process_chaining_results(
    simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
    results_type='simple', chatbot_id=chatbot_id, 
    save_df=True, save_chatbot=False
    )
simple_summaries_dict[chatbot_id]

New results shape: (40, 6)
simple summaries dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/prompt_chain_simple_summaries_simple_2023-04-19_1438.sav
Time completed: 2023-04-19 14:38:42.109518
	Object saved as pickle
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/prompt_chain_simple_summaries_simple_2023-04-19_1438.csv
Time completed: 2023-04-19 14:38:42.135039
	DataFrame saved as CSV
DataFrame pickled.


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,simple summary choice,audience,simplify task,full simplify task,simple summary
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younger individua...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A recent study found that middle-aged men who regularly do resistance training don't have a higher risk of muscle soreness or damage compared to younger men. This means that if you're middle-aged and you work out regularly, you don't have to worry about hurting your muscles more than younger peo..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A recent study found no differences in recovery response between young and middle-aged men following resistance exercise. Additionally, there were no differences in inflammatory or muscle damage response between the two groups. The study suggests that changes in muscle performance may begin duri...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A new study found that young and middle-aged men have similar recovery responses after doing weightlifting exercises. The study also found that there were no differences in how their muscles responded to the exercise. This means that starting in middle age, our muscles may start to change, but d..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,3,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A study comparing young and middle-aged men in the recovery response from resistance exercise found no significant differences in performance measures, muscle pain or soreness, inflammatory or muscle damage response. Differences in strength levels were observed between age groups, but recovery f...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A recent study looked at how young and middle-aged men recover from exercise. The researchers found that there were no big differences in how the two groups felt after 

# Run iteration 1.2: Add empty columns

In [112]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]

    if empty_columns:
        for column_name, column_number in empty_columns.items():
            new_results.insert(loc=column_number, column=column_name, value='')
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])

    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

empty_columns = {
    'original summary content rating': 9,
    'original summary language rating': 10,
    'top summary': 11
}
chatbot_id = iteration_id
save_df = False
simple_summaries_dict = process_chaining_results(
    simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
    results_type='simple', empty_columns=empty_columns,
    chatbot_id=chatbot_id, 
    save_df=save_df, save_chatbot=False
    )
simple_summaries_dict[chatbot_id]

New results shape: (40, 6)
simple summaries dataframe shape: (40, 17)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'original summary content rating', 'original summary language rating', 'top summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,original summary content rating,original summary language rating,top summary,simple summary choice,audience,simplify task,full simplify task,simple summary
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younger individua...",,,,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A recent study found that middle-aged men who regularly do resistance training don't have a higher risk of muscle soreness or damage compared to younger men. This means that if you're middle-aged and you work out regularly, you don't have to worry about hurting your muscles more than younger peo..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A recent study found no differences in recovery response between young and middle-aged men following resistance exercise. Additionally, there were no differences in inflammatory or muscle damage response between the two groups. The study suggests that changes in muscle performance may begin duri...",,,,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A new study found that young and middle-aged men have similar recovery responses after doing weightlifting exercises. The study also found that there were no differences in how their muscles responded to the exercise. This means that starting in middle age, our muscles may start to change, but d..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,3,You are an expert at science communication.,gpt-3.5-turbo,"Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furth...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A study comparing young and middle-aged men in the recovery response from resistance exercise found no significant differences in performance measures, muscle pain or soreness, inflammatory or muscle damage response. Differences in strength levels were observed between age groups, but recovery f...",,,,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,"A recent study looked at how young and middle-aged men recover from exercise. The 

# Run iteration 1.3: merge simple summaries with added relevance
Number of rows will increase by an order equal to the number of relevance prompt (i.e. double the rows if there are 2 relevance prompts)

In [161]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]

    if empty_columns:
        for column_name, column_number in empty_columns.items():
            new_results.insert(loc=column_number, column=column_name, value='')
    print(f'{"** simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])

    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

def merge_chaining_results(
        simple_summaries_dict, relevance_dict, iteration_id, 
        empty_columns=None, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the summaries, simple summaries, added relevance into a single DataFrame. 
    """
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    simple_summary_df = process_chaining_results(
        simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
        results_type='simple', empty_columns=empty_columns,
        chatbot_id=chatbot_id, 
        save_df=save_df, save_chatbot=False
        )[iteration_id]
    relevance_df = process_chaining_results(
        relevance_dict, qna_dict, chatbot_dict, iteration_id, 
        results_type='relevance', chatbot_id=chatbot_id, 
        save_df=save_df, save_chatbot=save_chatbot
        )[iteration_id]
    
    # Identify common columns
    common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
    common_columns.remove('audience')
    # common_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']

    print('Merging on common columns:', common_columns)
    # Perform outer merge using common columns
    merged_df = simple_summary_df.merge(
        relevance_df, how='left', suffixes=(' simplify', ' relevance'),
        left_on=common_columns, right_on=common_columns,
        validate=validate
        )
    print('Merged dataframe shape:', merged_df.shape)
    print([column for column in merged_df.columns])
    # for column in merged_df.columns:
    #     print(column, len(merged_df[column].unique()))
    return merged_df



empty_columns = {
    'original summary content rating': 9,
    'original summary language rating': 10,
    'top summary': 11
}
chatbot_id = iteration_id
save_df = False

merge_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']
merge_chaining_results(simple_summaries_dict, relevance_dict, iteration_id, chatbot_id=chatbot_id, save_df=save_df).head()

New results shape: (40, 6)
** simple summaries dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']
New results shape: (40, 6)
added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
Merging on common columns: ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']
Merged dataframe shape: (80, 19)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience simplify', 'simplify task', 'full simplif

,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,simple summary choice,audience simplify,simplify task,full simplify task,simple summary,relevance choice,audience relevance,relevance task,full relevance task,relevance statement
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A recent study found that middle-aged men who regularly do resistance training don't have a high...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors who regularly engage in resistance training, this study suggests that they may not e..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A recent study found that middle-aged men who regularly do resistance training don't have a high...,1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for people who enjoy sports,"This study is great news for middle-aged athletes who regularly engage in resistance training, a..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A new study found that young and middle-aged men have similar recovery responses after doing wei...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors, this study highlights the importance of engaging in resistance training to maintain..."
3,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A new study found that young and middle-aged men have similar recovery responses after doing wei...,1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for people who enjoy sports,"For sports enthusiasts, this study highlights the importance of incorporating re

In [151]:
merge_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']
simple_summaries_dict[chatbot_id].tail(10).merge(
    relevance_dict[chatbot_id].tail(10),
    on=merge_columns
)

,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,simple summary choice,audience_x,simplify task,full simplify task,simple summary,relevance choice,audience_y,relevance task,full relevance task,relevance statement
0,Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Simi...,1,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Simi...,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworke...,"Hey, did you know that a recent study investigated the effects of aging on muscle soreness and m...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,Did you know that a recent study looked at how getting older affects muscle soreness and damage ...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"This study is particularly relevant for seniors who engage in resistance exercise, as it suggest..."
1,Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Simi...,1,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Simi...,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworke...,"Hey, did you know that a recent study investigated the effects of aging on muscle soreness and m...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,Did you know that a recent study looked at how getting older affects muscle soreness and damage ...,1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for people who enjoy sports,This study provides valuable insights for athletes of all ages who are looking to optimize their...
2,Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Simi...,1,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Simi...,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworke...,"Hey, did you know that a recent study investigated the effects of aging on muscle soreness and m...",1,people without a science background,Use language appropriate for,Use language appropriate for for people without a science background,Hey there! Did you know that a recent study looked at how getting older affects muscle soreness ...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"This study is particularly relevant for seniors who engage in resistance exercise, as it suggest..."
3,Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Simi...,1,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Simi...,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworke...,"Hey, did you know that a recent study investigated the effects of aging on muscle soreness and m...",1,people without a science background,Use language appropriate for,Use language appropriate for for people without a science

In [150]:
simple_summaries_dict[chatbot_id][merge_columns] == relevance_dict[chatbot_id][merge_columns]

,summary,full summarization task,summarization task,prep step,model,choice,article_title,text,system_role
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True


In [131]:
simple_summaries_dict[chatbot_id].shape

(40, 17)

# Run iteration 1.4: works with pivot table so that no 2 rows have the same original summary

In [192]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]

    if empty_columns:
        for column_name, column_number in empty_columns.items():
            new_results.insert(loc=column_number, column=column_name, value='')
    print(f'{"** simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])

    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

def merge_chaining_results(
        simple_summaries_dict, relevance_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the summaries, simple summaries, added relevance into a single DataFrame. 
    """
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    simple_summary_df = process_chaining_results(
        simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
        results_type='simple', empty_columns=empty_columns,
        chatbot_id=chatbot_id, 
        save_df=save_df, save_chatbot=False
        )[iteration_id]
    relevance_df = process_chaining_results(
        relevance_dict, qna_dict, chatbot_dict, iteration_id, 
        results_type='relevance', chatbot_id=chatbot_id, 
        save_df=save_df, save_chatbot=save_chatbot
        )[iteration_id]
    
    if pivot == False:
        validate=None
        # Identify common columns
        common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
        common_columns.remove('audience')
        # common_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']

        print('Merging on common columns:', common_columns)
        # Perform outer merge using common columns
        merged_df = simple_summary_df.merge(
            relevance_df, how='left', suffixes=(' simplify', ' relevance'),
            on=common_columns,
            validate=validate
            )
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index='summary'
        )
        merged_df = simple_summary_df.merge(
            pivot_df, how='left', suffixes=(' simplify', ' relevance'),
            on='summary',
            validate='m:1' if validate else None
        )
    print('Merged dataframe shape:', merged_df.shape)
    print([column for column in merged_df.columns])
    return merged_df



empty_columns = {
    'original summary content rating': 9,
    'original summary language rating': 10,
    'top summary': 11
}
chatbot_id = iteration_id
save_df = False
pivot=True

merge_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']
merge_chaining_results(
    simple_summaries_dict, relevance_dict, iteration_id, chatbot_id=chatbot_id, save_df=save_df,
    pivot=pivot, validate=True
    ).head()

New results shape: (40, 6)
** simple summaries dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']
New results shape: (40, 6)
added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
Merged dataframe shape: (40, 16)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary', 'people who enjoy sports', 'seniors']


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,simple summary choice,audience,simplify task,full simplify task,simple summary,people who enjoy sports,seniors
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A recent study found that middle-aged men who regularly do resistance training don't have a high...,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...","For seniors who regularly engage in resistance training, this study suggests that they may not e..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A new study found that young and middle-aged men have similar recovery responses after doing wei...,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...","For seniors, this study highlights the importance of engaging in resistance training to maintain..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study comparing young and middle-aged men in the recovery response from resistance exercise fo...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A recent study looked at how young and middle-aged men recover from exercise. The researchers fo...,"For sports enthusiasts, this study suggests that age should not be a limiting factor in recovery...",This study is particularly relevant for seniors as it suggests that recreational resistance trai...
3,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"""New study finds no difference in recovery response from resistance exercise between young and m...",1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A new study found that middle-aged men who do resistance exercise (like lifting weights) don't h...,"This study is particularly relevant for athletes who engage in resistance training, as it sugges...","For seniors, resistance training can be an effective way to maintain muscle mass and strength, w..."
4,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,5,You are an expert at scien

# Run iteration 1.5

In [202]:
import string
alphabet_dict = {char:idx+1 for idx, char in enumerate(string.ascii_uppercase)}
alphabet_dict['s'.upper()]

19

In [216]:
[letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'AA',
 'AB',
 'AC',
 'AD',
 'AE',
 'AF',
 'AG',
 'AH',
 'AI',
 'AJ',
 'AK',
 'AL',
 'AM',
 'AN',
 'AO',
 'AP',
 'AQ',
 'AR',
 'AS',
 'AT',
 'AU',
 'AV',
 'AW',
 'AX',
 'AY',
 'AZ']

In [253]:

import string
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]

    if empty_columns:
        for column_name, column_number in empty_columns.items():
            new_results.insert(loc=column_number, column=column_name, value='')
    print(f'** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])

    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

def merge_chaining_results(
        simple_summaries_dict, relevance_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the summaries, simple summaries, added relevance into a single DataFrame. 
    """
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    simple_summary_df = process_chaining_results(
        simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
        results_type='simple', empty_columns=None,
        chatbot_id=chatbot_id, 
        save_df=False, save_chatbot=False
        )[iteration_id]
    relevance_df = process_chaining_results(
        relevance_dict, qna_dict, chatbot_dict, iteration_id, 
        results_type='relevance', chatbot_id=chatbot_id, 
        save_df=False, save_chatbot=save_chatbot
        )[iteration_id]
    
    if pivot == False:
        validate=None
        # Identify common columns
        common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
        common_columns.remove('audience')

        print('Merging on common columns:', common_columns)
        merged_df = simple_summary_df.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=common_columns,
            validate=validate
            )
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['summary', 'relevance task']
        ).reset_index()
        merged_df = simple_summary_df.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=['summary'],
            validate='m:1' if validate else None
        )
    if empty_columns:
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name in empty_columns.keys():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] 
            # print(f'Inserting column "{column_name}" to column {empty_column_loc}')
            print(f'{column_name}', end=', ')
            merged_df.insert(loc=empty_column_loc, column=column_name, value='')
    if save_df:
        try:
            save_output(
                merged_df, description=f'prompt_chain_summaries_and_relevance',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')

    print('\nMerged DataFrame shape:', merged_df.shape)
    print('Merged DataFrame columns: ', [column for column in merged_df.columns])
    return merged_df


# # first_empty
# empty_columns = {
#     'original summary content rating': 9,
#     'original summary language rating': 10,
#     'top summary': 11,
#     'simple summary content rating': empty_columns['top summary'] + 3,
#     'simple summary language rating': empty_columns['simple summary content rating'] + 1,
#     'top simple summary': empty_columns['simple summary language rating'] + 1,
# }
empty_columns = {
    'original summary content rating': 'J',
    'original summary language rating': 'K',
    'top summary': 'l',
    'simple summary content rating': 'r',
    'simple summary language rating': 's',
    'top simple summary': 't',
    'full add relevance task': 'u',
    'added relevance content rating': 'x',
    'added relevance language rating': 'y',
}
chatbot_id = iteration_id
save_df = True
pivot=True

merge_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']
merge_chaining_results(
    simple_summaries_dict, relevance_dict, iteration_id, chatbot_id=chatbot_id, save_df=save_df,
    empty_columns=empty_columns,
    pivot=pivot, validate=True
    )

New results shape: (40, 6)
** simple summaries dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']
New results shape: (40, 6)
** added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
Inserting empty columns...
	original summary content rating, original summary language rating, top summary, simple summary content rating, simple summary language rating, top simple summary, full add relevance task, added relevance content rating, added relevance language rating, File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/prompt_chain_summar

,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,original summary content rating,...,simple summary,simple summary content rating,simple summary language rating,top simple summary,full add relevance task,relevance task,people who enjoy sports,added relevance content rating,added relevance language rating,seniors
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,,...,A recent study found that middle-aged men who regularly do resistance training don't have a high...,,,,,Add 1-2 sentences to make this relevant for,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...",,,"For seniors who regularly engage in resistance training, this study suggests that they may not e..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,,...,A recent study found that middle-aged men who regularly do resistance training don't have a high...,,,,,Add 1-2 sentences to make this relevant for,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...",,,"For seniors who regularly engage in resistance training, this study suggests that they may not e..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,,...,A new study found that young and middle-aged men have similar recovery responses after doing wei...,,,,,Add 1-2 sentences to make this relevant for,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...",,,"For seniors, this study highlights the importance of engaging in resistance training to maintain..."
3,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,,...,A new study found that young and middle-aged men have similar recovery responses after doing wei...,,,,,Add 1-2 sentences to make this relevant for,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...",,,"For seniors, this study highlights the importance of engaging in resistance training to maintain..."
4,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,3,You are an expert at science 

# Run iteration 1.6: This works

In [338]:

from collections import OrderedDict
import string
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]

    if empty_columns:
        for column_name, column_number in empty_columns.items():
            new_results.insert(loc=column_number, column=column_name, value='')
    print(f'** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])

    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

def merge_chaining_results(
        simple_summaries_dict, relevance_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the summaries, simple summaries, added relevance into a single DataFrame. 
    """
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    simple_summary_df = process_chaining_results(
        simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
        results_type='simple', empty_columns=None,
        chatbot_id=chatbot_id, 
        save_df=False, save_chatbot=False
        )[iteration_id]
    relevance_df = process_chaining_results(
        relevance_dict, qna_dict, chatbot_dict, iteration_id, 
        results_type='relevance', chatbot_id=chatbot_id, 
        save_df=False, save_chatbot=save_chatbot
        )[iteration_id]
    
    if pivot == False:
        validate=None
        # Identify common columns
        common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
        common_columns.remove('audience')

        print('Merging on common columns:', common_columns)
        merged_df = simple_summary_df.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=common_columns,
            validate=validate
            )
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['summary', 'relevance task']
        ).reset_index()
        merged_df = simple_summary_df.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=['summary'],
            validate='m:1' if validate else None
        )
    if empty_columns:
        print('\nColumns before adding empty columns:', [column for column in merged_df.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = OrderedDict({char:idx for idx, char in enumerate(spreadsheet_columns)})
        for column_name in empty_columns.keys():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            merged_df.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')

        merged_df.columns = [f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(merged_df.columns)]

    if save_df:
        try:
            save_output(
                merged_df, description=f'prompt_chain_summaries_and_relevance',
                csv_path=csv_path, pickle_path=pickle_path)
            print('DataFrame pickled.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')

    print('\nMerged DataFrame shape:', merged_df.shape)
    print('Merged DataFrame columns: ', [column for column in merged_df.columns])
    return merged_df


empty_columns = OrderedDict({
    'original summary content rating': 'k',
    'original summary language rating': 'l',
    'top summary': 'm',
    'simple summary content rating': 's',
    'simple summary language rating': 't',
    'top simple summary': 'u',
    'full add relevance task': 'w',
    'added relevance content rating': 'y',
    'added relevance language rating': 'z',
    'add relevance task (seniors)': 'AA',
    'full add relevance task (seniors)': 'AB',
})
chatbot_id = iteration_id
save_df = True
pivot=True

merge_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']
merge_chaining_results(
    simple_summaries_dict, relevance_dict, iteration_id, chatbot_id=chatbot_id, save_df=save_df,
    empty_columns=empty_columns,
    pivot=pivot, validate=True
    )

New results shape: (40, 6)
** simple summaries dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']
New results shape: (40, 6)
** added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']

Columns before adding empty columns: ['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary', 'relevance task', 'people who enjoy sports', 'seniors']
Inserting empty columns...
	K (9): original summary content rating, L (10): or

,B: article_title,C: choice,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,...,T: simple summary language rating,U: top simple summary,V: relevance task,W: full add relevance task,X: people who enjoy sports,Y: added relevance content rating,Z: added relevance language rating,AA: add relevance task (seniors),AB: full add relevance task (seniors),AC: seniors
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,,...,,,Add 1-2 sentences to make this relevant for,,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...",,,,,"For seniors who regularly engage in resistance training, this study suggests that they may not e..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,,...,,,Add 1-2 sentences to make this relevant for,,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...",,,,,"For seniors who regularly engage in resistance training, this study suggests that they may not e..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,,...,,,Add 1-2 sentences to make this relevant for,,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...",,,,,"For seniors, this study highlights the importance of engaging in resistance training to maintain..."
3,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,,...,,,Add 1-2 sentences to make this relevant for,,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...",,,,,"For seniors, this study highlights the importance of engaging in resistance training to maintain..."
4,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study comparing young and 

# Run 1.7 Embed empty_columns dictionary within function

In [360]:

# from collections import OrderedDict
import string
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]

    if empty_columns:
        if (type(empty_columns) != dict) & (type(empty_columns) != OrderedDict):
            empty_columns = {
                'original summary content rating': 'k',
                'original summary language rating': 'l',
                'top summary': 'm',
                'simple summary content rating': 's',
                'simple summary language rating': 't',
                'top simple summary': 'u',
                'full add relevance task': 'w',
                'added relevance content rating': 'y',
                'added relevance language rating': 'z',
                'add relevance task (seniors)': 'AA',
                'full add relevance task (seniors)': 'AB',
            }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])

    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

def merge_chaining_results(
        simple_summaries_dict, relevance_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the summaries, simple summaries, added relevance into a single DataFrame. 
    Parameters:
        - simple_summaries_dict: dictionary of simple summaries DataFrames
        - relevance_dict: dictionary of added relevance DataFrames
        - iteration_id: iteration ID of the chatbot
        - empty_columns (dict, OrderedDict, int, bool): dictionary of empty columns to insert into the DataFrame. 
            Keys are the column names, values are the column letters in the spreadsheet.
            If True but dictionary is not passed, a default dictionary will be used
            based on Google sheet as of 2023-04-19.
        - pivot (bool): whether to pivot the DataFrame so that 
            multiple added relevance statements are in the same row.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int): ID of the chatbot to use if different from iteration_id.
        - save_df (bool): whether to save the DataFrame.
        - save_chatbot (bool): whether to save the chatbot.
        - csv_path (str): path to save the csv file. If None, no CSV is saved.
        - pickle_path (str): path to save the pickle file. If None, no pickle is saved.

    """
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    simple_summary_df = process_chaining_results(
        simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
        results_type='simple', empty_columns=None,
        chatbot_id=chatbot_id, 
        save_df=False, save_chatbot=False,
        csv_path=csv_path, pickle_path=pickle_path, json_path=json_path
        )[iteration_id]
    relevance_df = process_chaining_results(
        relevance_dict, qna_dict, chatbot_dict, iteration_id, 
        results_type='relevance', chatbot_id=chatbot_id, 
        save_df=False, save_chatbot=save_chatbot,
        csv_path=csv_path, pickle_path=pickle_path, json_path=json_path
        )[iteration_id]
    
    if pivot == False:
        validate=None
        # Identify common columns
        common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
        common_columns.remove('audience')

        print('Merging on common columns:', common_columns)
        merged_df = simple_summary_df.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=common_columns,
            validate=validate
            )
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['summary', 'relevance task']
        ).reset_index()
        merged_df = simple_summary_df.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=['summary'],
            validate='m:1' if validate else None
        )
    if empty_columns:
        print(type(empty_columns)) #### delete this
        if (type(empty_columns) != dict) & (type(empty_columns) != OrderedDict):
            empty_columns = {
                'original summary content rating': 'k',
                'original summary language rating': 'l',
                'top summary': 'm',
                'simple summary content rating': 's',
                'simple summary language rating': 't',
                'top simple summary': 'u',
                'full add relevance task': 'w',
                'added relevance content rating': 'y',
                'added relevance language rating': 'z',
                'add relevance task (seniors)': 'AA',
                'full add relevance task (seniors)': 'AB',
            }
        print('\nColumns before adding empty columns:', [column for column in merged_df.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name in empty_columns.keys():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            merged_df.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')

        merged_df.columns = [f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(merged_df.columns)]

    if save_df:
        try:
            save_output(
                merged_df, description=f'prompt_chain_summaries_and_relevance',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    
    print('\nMerged DataFrame shape:', merged_df.shape)
    print('Merged DataFrame columns: ', [column for column in merged_df.columns])
    return merged_df


empty_columns = {
    'original summary content rating': 'k',
    'original summary language rating': 'l',
    'top summary': 'm',
    'simple summary content rating': 's',
    'simple summary language rating': 't',
    'top simple summary': 'u',
    'full add relevance task': 'w',
    'added relevance content rating': 'y',
    'added relevance language rating': 'z',
    'add relevance task (seniors)': 'AA',
    'full add relevance task (seniors)': 'AB',
}
chatbot_id = iteration_id
save_df = False
save_chatbot = True
pivot=True

merge_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']
merge_chaining_results(
    simple_summaries_dict, relevance_dict, iteration_id, chatbot_id=chatbot_id, save_df=save_df,
    empty_columns=OrderedDict({'hello': 'd'}),
    pivot=pivot, validate=True, save_chatbot=save_chatbot,
    json_path=r'G:\Shared drives\content summarization\raw outputs\JSON'
    )

New results shape: (40, 6)
** simple summaries dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']
New results shape: (40, 6)
** added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
Saving Chaining object (chatbot)...
text1_prompt00
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text1_prompt01
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text2_prompt00
	Attribute

,B: article_title,C: choice,D: hello,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: full summarization task,K: summary,L: simple summary choice,M: audience,N: simplify task,O: full simplify task,P: simple summary,Q: relevance task,R: people who enjoy sports,S: seniors
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A recent study found that middle-aged men who regularly do resistance training don't have a high...,Add 1-2 sentences to make this relevant for,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...","For seniors who regularly engage in resistance training, this study suggests that they may not e..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,1,people without a science background,Use language appropriate for,Use language appropriate for for people without a science background,A recent study found that middle-aged men who regularly do resistance training don't have a high...,Add 1-2 sentences to make this relevant for,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...","For seniors who regularly engage in resistance training, this study suggests that they may not e..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A new study found that young and middle-aged men have similar recovery responses after doing wei...,Add 1-2 sentences to make this relevant for,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...","For seniors, this study highlights the importance of engaging in resistance training to maintain..."
3,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,1,people without a science background,Use language appropriate for,Use language appropriate for for people without a science background,A new study found that young and middle-aged men have similar recovery responses after doing wei...,Add 1-2 sentences to make this relev

In [358]:
simple_summaries_dict[iteration_id]['full simplify task'].unique()

array(['Use language appropriate for for a lay audience',
       'Use language appropriate for for people without a science background'],
      dtype=object)

# update empty columns dict

In [ ]:

# from collections import OrderedDict
import string
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """
    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in 
            more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the 
                values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    new_results = new_results[columns]

    if empty_columns:
        if (type(empty_columns) != dict) & (type(empty_columns) != OrderedDict):
            empty_columns = {
                'original summary content rating': 'k',
                'original summary language rating': 'l',
                'top summary': 'm',
                'simple summary content rating': 's',
                'simple summary language rating': 't',
                'top simple summary': 'u',
                'full add relevance task': 'w',
                'added relevance content rating': 'y',
                'added relevance language rating': 'z',
                'add relevance task (seniors)': 'AA',
                'full add relevance task (seniors)': 'AB',
            }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])

    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

def merge_chaining_results(
        simple_summaries_dict, relevance_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the summaries, simple summaries, added relevance into a single DataFrame. 
    Parameters:
        - simple_summaries_dict: dictionary of simple summaries DataFrames
        - relevance_dict: dictionary of added relevance DataFrames
        - iteration_id: iteration ID of the chatbot
        - empty_columns (dict, OrderedDict, int, bool): dictionary of empty columns to insert into the DataFrame. 
            Keys are the column names, values are the column letters in the spreadsheet.
            If True but dictionary is not passed, a default dictionary will be used
            based on Google sheet as of 2023-04-19.
        - pivot (bool): whether to pivot the DataFrame so that 
            multiple added relevance statements are in the same row.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int): ID of the chatbot to use if different from iteration_id.
        - save_df (bool): whether to save the DataFrame.
        - save_chatbot (bool): whether to save the chatbot.
        - csv_path (str): path to save the csv file. If None, no CSV is saved.
        - pickle_path (str): path to save the pickle file. If None, no pickle is saved.

    """
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    simple_summary_df = process_chaining_results(
        simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
        results_type='simple', empty_columns=None,
        chatbot_id=chatbot_id, 
        save_df=False, save_chatbot=False,
        csv_path=csv_path, pickle_path=pickle_path, json_path=json_path
        )[iteration_id]
    relevance_df = process_chaining_results(
        relevance_dict, qna_dict, chatbot_dict, iteration_id, 
        results_type='relevance', chatbot_id=chatbot_id, 
        save_df=False, save_chatbot=save_chatbot,
        csv_path=csv_path, pickle_path=pickle_path, json_path=json_path
        )[iteration_id]
    
    if pivot == False:
        validate=None
        # Identify common columns
        common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
        common_columns.remove('audience')

        print('Merging on common columns:', common_columns)
        merged_df = simple_summary_df.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=common_columns,
            validate=validate
            )
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['summary', 'relevance task']
        ).reset_index()
        merged_df = simple_summary_df.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=['summary'],
            validate='m:1' if validate else None
        )
    if empty_columns:
        print(type(empty_columns)) #### delete this
        if (type(empty_columns) != dict) & (type(empty_columns) != OrderedDict):
            empty_columns = {
                'original summary content rating': 'k',
                'original summary language rating': 'l',
                'top summary': 'm',
                'simple summary content rating': 's',
                'simple summary language rating': 't',
                'top simple summary': 'u',
                'full add relevance task': 'w',
                'added relevance content rating': 'y',
                'added relevance language rating': 'z',
                'add relevance task (seniors)': 'AA',
                'full add relevance task (seniors)': 'AB',
            }
        print('\nColumns before adding empty columns:', [column for column in merged_df.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name in empty_columns.keys():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            merged_df.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')

        merged_df.columns = [f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(merged_df.columns)]

    if save_df:
        try:
            save_output(
                merged_df, description=f'prompt_chain_summaries_and_relevance',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    
    print('\nMerged DataFrame shape:', merged_df.shape)
    print('Merged DataFrame columns: ', [column for column in merged_df.columns])
    return merged_df


empty_columns = {
    'original summary content rating': 'k',
    'original summary language rating': 'l',
    'top summary': 'm',
    'simple summary content rating': 's',
    'simple summary language rating': 't',
    'top simple summary': 'u',
    'full add relevance task': 'w',
    'added relevance content rating': 'y',
    'added relevance language rating': 'z',
    'add relevance task (seniors)': 'AA',
    'full add relevance task (seniors)': 'AB',
}
chatbot_id = iteration_id
save_df = False
save_chatbot = True
pivot=True

merge_columns = ['summary', 'full summarization task', 'summarization task', 'prep step', 'model', 'choice', 'article_title', 'text', 'system_role']
merge_chaining_results(
    simple_summaries_dict, relevance_dict, iteration_id, chatbot_id=chatbot_id, save_df=save_df,
    empty_columns=OrderedDict({'hello': 'd'}),
    pivot=pivot, validate=True, save_chatbot=save_chatbot,
    json_path=r'G:\Shared drives\content summarization\raw outputs\JSON'
    )

New results shape: (40, 6)
** simple summaries dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']
New results shape: (40, 6)
** added relevance dataframe shape: (40, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
Saving Chaining object (chatbot)...
text1_prompt00
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text1_prompt01
	Attributes: text, system_role, temperature, max_tokens, model, qna, summaries_dict, article_title, response_regex, simple_summary_dict, relevance_dict, 
text2_prompt00
	Attribute

,B: article_title,C: choice,D: hello,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: full summarization task,K: summary,L: simple summary choice,M: audience,N: simplify task,O: full simplify task,P: simple summary,Q: relevance task,R: people who enjoy sports,S: seniors
0,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A recent study found that middle-aged men who regularly do resistance training don't have a high...,Add 1-2 sentences to make this relevant for,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...","For seniors who regularly engage in resistance training, this study suggests that they may not e..."
1,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,1,,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A study found no differences in recovery response between young and middle-aged men after high-v...,1,people without a science background,Use language appropriate for,Use language appropriate for for people without a science background,A recent study found that middle-aged men who regularly do resistance training don't have a high...,Add 1-2 sentences to make this relevant for,"This study is great news for middle-aged athletes who regularly engage in resistance training, a...","For seniors who regularly engage in resistance training, this study suggests that they may not e..."
2,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,1,a lay audience,Use language appropriate for,Use language appropriate for for a lay audience,A new study found that young and middle-aged men have similar recovery responses after doing wei...,Add 1-2 sentences to make this relevant for,"For sports enthusiasts, this study highlights the importance of incorporating resistance trainin...","For seniors, this study highlights the importance of engaging in resistance training to maintain..."
3,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men,2,,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,A recent study found no differences in recovery response between young and middle-aged men follo...,1,people without a science background,Use language appropriate for,Use language appropriate for for people without a science background,A new study found that young and middle-aged men have similar recovery responses after doing wei...,Add 1-2 sentences to make this relev

# Unpickle Chaining object saved as dictionary

In [ ]:
pickle_path

# *End of Page*